In [ ]:
R.version

In [2]:
#install.packages("tidyverse")
install.packages("plyr")
install.packages("dplyr")
install.packages("readr")
install.packages("stringr")
install.packages("tidyr")
install.packages("tibble")
install.packages("reshape2")

also installing the dependency ‘Rcpp’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘pkgconfig’, ‘purrr’, ‘generics’, ‘magrittr’, ‘R6’, ‘tibble’, ‘tidyselect’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘bit’, ‘prettyunits’, ‘bit64’, ‘withr’, ‘progress’, ‘clipr’, ‘hms’, ‘vroom’, ‘cpp11’, ‘tzdb’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependency ‘stringi’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
#plyr before dplyr!
#library(tidyverse)

library(readr)
library(plyr)
library(dplyr)
library(stringr)
library(tidyr)
library(tibble)
library(reshape2)

In [3]:
#read-in "name", "expanded_in" and "compared_to" 
num = "2"
name = "Expanded in barley compared to maize"
#expanded_in = "Arabidopsis_thaliana"
#compared_to = c("Hordeum_vulgare", "Zea_mays")
#expanded_in = "Arabidopsis_thaliana"
#compared_to = c("Zea_mays", "Hordeum_vulgare")
expanded_in = c("Zea_mays", "Hordeum_vulgare")
compared_to = "Arabidopsis_thaliana"

In [4]:
OG.GC <- readr::read_tsv("../orthofinder/final-results/Orthogroups/Orthogroups.GeneCount.tsv")

Rows: 7621 Columns: 5
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): Orthogroup
dbl (4): Arabidopsis_thaliana, Hordeum_vulgare, Zea_mays, Total

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
#THis step is actually in the python functions; however here I I need a solution to test the downstream steps:
#I can use strsplit to split the ";" seperated columns
#strsplit creates a list and the [[1]] selects the first list item (we only have one, in this case).
#The result at this point is in list form, we can use "unlist" that works with the downstream steps.
#compared_to
#c_t_species <- unlist(strsplit(compared_to, ";"))
#compared_to
c_t_species <- compared_to
c_t_species

[1] "Arabidopsis_thaliana"

In [8]:
#get() function solves my problems of using the character functions inside dplyr
#expanded_OGs <- OG.GC
#for (i in c_t_species) {
#    expanded_OGs <- expanded_OGs %>% dplyr::filter(get(expanded_in) > 3*(get(i)))
#    }
#expanded_OGs

In [6]:
print("hello")

[1] "hello"


In [ ]:
##keeping in line with smaller scripts and modularity:

#1) save Orthogroups for this hypothesis as first result; TEA directory; subd. GFEs ;subd. hypothesis number

#2)parallel seperate rule
#getting all BLAST hits and substituting for actual OG names

#3) use hypothesis wildcard and new R script to perform BLAST filtering
# -> write these results to file again

#4) snakemake muscle, gblocks and fastTree (alternative programmes?)

#5) load data into R, plot with ggtree; 
#-> here, stability and cleanup

#6) and then all the other stuff

In [7]:
#concatenate all BLAST seaches to one file and create parseable tibble with read_delim
#could add that only BLAST searches of our species in this hypothesis are used - toDO for later
#easiest and efficient way is to bind outside of the loop
#do.call is a nice way rbind(bind_rows[[1]], bind_rows[[2]], ...)
datalist = list()
for (i in Sys.glob("../orthofinder/search/*.txt")) {
    datalist[[i]] <- readr::read_delim(file = i,
                                delim = "\t",
                                col_names = c(
                                    "qseqid",
                                    "sseqid",
                                    "pident",
                                    "length",
                                    "mismatch",
                                    "gapopen",
                                    "qstart",
                                    "qend",
                                    "sstart",
                                    "send",
                                    "evalue",
                                    "bitscore"),
                                col_types = c(
                                    qseqid = col_character(),
                                    sseqid = col_character(),
                                    pident = col_double(),
                                    length = col_double(),
                                    mismatch = col_double(),
                                    gapopen = col_double(),
                                    qstart = col_double(),
                                    qend = col_double(),
                                    sstart = col_double(),
                                    send = col_double(),
                                    evalue = col_double(),
                                    bitscore = col_double()
                                                )
                                )
                                                }

all_BLAST <- base::do.call(bind_rows, datalist)

In [10]:
head(all_BLAST)

qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0_0,0_0,100,232,0,0,1,232,1,232,5.55e-165,452
0_0,0_1,100,232,0,0,1,232,1,232,5.55e-165,452
0_0,0_2,100,232,0,0,1,232,1,232,5.55e-165,452
0_0,0_3,100,232,0,0,1,232,1,232,5.55e-165,452
0_1,0_0,100,232,0,0,1,232,1,232,5.55e-165,452
0_1,0_1,100,232,0,0,1,232,1,232,5.55e-165,452


In [8]:
#read-in the conversion table: orthofinder sequence IDs and corresponding actual gene/protein names
#with "str_remove_all" I also easily get rid of the ":"
conversion <- readr::read_delim(file = "../orthofinder/SequenceIDs.txt",
                delim = " ",
                col_names = c("seqid", "name"),
                col_types = c(
                   seqid = col_character(),
                   name = col_character())               
               )
conversion <- conversion %>%  dplyr::mutate(seqid = str_remove_all(seqid, ":"))

Warning message:
“One or more parsing issues, see `problems()` for details”


In [13]:
conversion

seqid,name,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
0_0,AT2G27490.4,pep,chromosome:TAIR10:2:11748087:11749153:-1,gene:AT2G27490,transcript:AT2G27490.4,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:COAE,description:Dephospho-CoA,kinase,[Source:UniProtKB/Swiss-Prot;Acc:Q9ZQH0]
0_1,AT2G27490.1,pep,chromosome:TAIR10:2:11748015:11750488:-1,gene:AT2G27490,transcript:AT2G27490.1,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:COAE,description:Dephospho-CoA,kinase,[Source:UniProtKB/Swiss-Prot;Acc:Q9ZQH0]
0_2,AT2G27490.3,pep,chromosome:TAIR10:2:11748087:11750488:-1,gene:AT2G27490,transcript:AT2G27490.3,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:COAE,description:Dephospho-CoA,kinase,[Source:UniProtKB/Swiss-Prot;Acc:Q9ZQH0]
0_3,AT2G27490.2,pep,chromosome:TAIR10:2:11748015:11750488:-1,gene:AT2G27490,transcript:AT2G27490.2,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:COAE,description:Dephospho-CoA,kinase,[Source:UniProtKB/Swiss-Prot;Acc:Q9ZQH0]
0_4,AT2G17370.1,pep,chromosome:TAIR10:2:7549702:7552260:1,gene:AT2G17370,transcript:AT2G17370.1,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:HMG2,description:3-hydroxy-3-methylglutaryl-coenzyme,A,reductase 2 [Source:UniProtKB/Swiss-Prot;Acc:P43256]
0_5,AT2G34090.3,pep,chromosome:TAIR10:2:14396283:14399275:-1,gene:AT2G34090,transcript:AT2G34090.3,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:MEE18,description:maternal,effect,embryo arrest 18 [Source:TAIR;Acc:AT2G34090]
0_6,AT2G34090.2,pep,chromosome:TAIR10:2:14396178:14399265:-1,gene:AT2G34090,transcript:AT2G34090.2,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:MEE18,description:maternal,effect,embryo arrest 18 [Source:TAIR;Acc:AT2G34090]
0_7,AT2G34090.1,pep,chromosome:TAIR10:2:14396016:14399265:-1,gene:AT2G34090,transcript:AT2G34090.1,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:MEE18,description:maternal,effect,embryo arrest 18 [Source:TAIR;Acc:AT2G34090]
0_8,AT2G34090.4,pep,chromosome:TAIR10:2:14396283:14399275:-1,gene:AT2G34090,transcript:AT2G34090.4,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:MEE18,description:maternal,effect,embryo arrest 18 [Source:TAIR;Acc:AT2G34090]


In [9]:
#create new columns with the actual gene/protein names
all_BLAST_reformatted <- all_BLAST %>% dplyr::left_join(conversion, by = c("qseqid" = "seqid"))
all_BLAST_reformatted <- all_BLAST_reformatted %>% dplyr::left_join(conversion, by = c("sseqid" = "seqid"))
#position after qseqid and sseqid respectively
all_BLAST_reformatted <- all_BLAST_reformatted %>% dplyr::relocate(name.x, .after = qseqid)
all_BLAST_reformatted <- all_BLAST_reformatted %>% dplyr::relocate(name.y, .after = sseqid)
#rename to qseqid/sseqid_name
all_BLAST_reformatted <- dplyr::rename(all_BLAST_reformatted,  qseqid_name = name.x)
all_BLAST_reformatted <- dplyr::rename(all_BLAST_reformatted, sseqid_name = name.y)

In [ ]:
#write to file; for reference
#write_delim(as.data.frame(foo), "test.tsv", delim = "\t")

In [10]:
#read-in Orthogroups.txt old!!
orthogroups <- readr::read_delim(file = "../orthofinder/final-results/Orthogroups/Orthogroups.txt",
                          delim = ":",
                          col_names = c("OG", "genes"),
                          col_types = c(
                              OG = col_character(),
                              genes = col_character())
                         )

In [11]:
#read-in Orthogroups.txt new!!
#expanded OG_s also have to be computed based on this..
ph_orthogroups <- readr::read_delim(file = "../orthofinder/final-results/Phylogenetic_Hierarchical_Orthogroups/N0.tsv",
                          delim = "\t",
#                          col_names = c("HOG", "genes"),
#                          col_types = c(
#                              OG = col_character(),
#                              genes = col_character())
                         )
#head(ph_orthogroups)
#length(unlist(strsplit(ph_orthogroups[ph_orthogroups$HOG == "N0.HOG0000103",]$Arabidopsis_thaliana, ",")))
#length(unlist(strsplit(ph_orthogroups["N0.HOG0000103",]$Arabidopsis_thaliana, ",")))

#get() function solves my problems of using the character functions inside dplyr

#colnames(ph_orthogroups)
#compared_to
#length(unlist(strsplit(ph_orthogroups[ph_orthogroups[[compared_to]], ",")))
all_species <- c(expanded_in, compared_to)
#all_species[1]
#all_species[2]

#create dataframe with numbers of genes per PHOG
#we combine expanded_in and compared_to vectors to easily compute for everything we need

HOG_df <- setNames(base::data.frame(matrix(ncol = 1 + length(all_species), nrow = 0)),
                  c("HOG", all_species))

for (i in 1:nrow(ph_orthogroups)) {
    
    row = 0
    
    #print(ph_orthogroups[i,]$HOG)
    row <- c(ph_orthogroups[i,]$HOG)
    
        for (j in c(all_species)) {
            if (is.na(ph_orthogroups[i,][[j]]))
            {
                test = 0
                row <- c(row, test)
            }
            else 
            {
                test = length(unlist(strsplit(ph_orthogroups[i,][[j]], ",")))
                row <- c(row, test)
            }
}
    HOG_df[i,] <- row
} 

HOG_tibble <- as_tibble(HOG_df)

for (k in 1:length(all_species)) {
    o = all_species[k]
    HOG_tibble[[o]] <- as.numeric(HOG_tibble[[o]])
}
head(HOG_tibble)

#create "copy" as HOG_tibble is currently modified in the upcoming step
HOG_tibble_complete <- HOG_tibble


Rows: 7656 Columns: 6
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): HOG, OG, Gene Tree Parent Clade, Arabidopsis_thaliana, Hordeum_vulg...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


HOG,Zea_mays,Hordeum_vulgare,Arabidopsis_thaliana
<chr>,<dbl>,<dbl>,<dbl>
N0.HOG0000000,116,0,0
N0.HOG0000001,103,0,1
N0.HOG0000002,104,0,0
N0.HOG0000003,0,100,0
N0.HOG0000004,0,93,0
N0.HOG0000005,0,81,0


In [12]:
#head(my_tibble)
#tail(my_tibble)
#expanded_HOGs
#head(ph_orthogroups[])
HOG_tibble_complete[HOG_tibble_complete$HOG=="N0.HOG0000783",]
HOG_tibble_complete[HOG_tibble_complete$HOG=="N0.HOG0000027",]

HOG,Zea_mays,Hordeum_vulgare,Arabidopsis_thaliana
<chr>,<dbl>,<dbl>,<dbl>
N0.HOG0000783,0,16,0


HOG,Zea_mays,Hordeum_vulgare,Arabidopsis_thaliana
<chr>,<dbl>,<dbl>,<dbl>
N0.HOG0000027,0,56,1


In [ ]:
# function has to reduce input dataset; this way each iteration (compared species) is included
# for all compared species first check for each HOG if expanded species is >= 3* 
# then keep rows in which expanded species has at least 3
# since 3*1 = 3, we can simply choose to keep all rows with at least 3 in expanded_in since this 
## is performed after the other filtering ;D
# this we should talk about... how to deal with 0 cases
# in any case having this as user input shoud be the goal
# with addition of the outer t loop now we can also have multiple expansion species ;D

#for (i in c_t_species) {
#        HOG_tibble <- HOG_tibble %>% dplyr::filter(get(expanded_in) >= 3*(get(i)))
#        HOG_tibble <- HOG_tibble %>% dplyr::filter(get(expanded_in) >= 3)
#        expanded_HOGs <- HOG_tibble
#    }

#for (t in expanded_in) {
#for (i in c_t_species) {
#        HOG_tibble <- HOG_tibble %>% dplyr::filter(get(expanded_in) >= expansion_factor*(get(i)))
#        HOG_tibble <- HOG_tibble %>% dplyr::filter(get(expanded_in) >= expansion_factor)
#    # this line will remove all HOGs with 0 cases in BOTH compared species
#    # this means, that for now, a multispecies comparison demands that
#    # for all compared_to species at least 1 gene has to be in the HOG!
#        HOG_tibble <- HOG_tibble %>% dplyr::filter(get(i) > 0)
#        expanded_HOGs <- HOG_tibble
#    }
#    }

In [19]:
#sessionInfo() 

R version 4.1.0 (2021-05-18)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS/LAPACK: /data/home/stoecker/miniconda3/lib/libopenblasp-r0.3.15.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] tibble_3.1.2  tidyr_1.1.3   stringr_1.4.0 dplyr_1.0.7   plyr_1.8.6   
[6] readr_1.4.0  

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.7        pillar_1.6.1      compiler_4.1.0    base64enc_0.1-3  
 [5] tools_4.1.0       digest_0.6.27     uuid_0.1-4        jsonlite_1.7.2   
 [9] evaluate_0.14     lifecycle_1.0.0   pkgconfig_2.0.3   rlang_0.4.11     
[13] IRdisplay_1.0     DBI_1.1.1         cli_2.5.0         rstudioapi_0.13  
[17] IRkernel_1.2      repr_1.1.3        generics_0.1.0    vctrs_0.3.8      
[21] hms_1.1.0         tidyselect_1.1.1  glue_1.4.2        R6_2.5.0         
[25] fansi_0.5.0       pbdZMQ_0.3-5      purrr_0.3.4       magrittr_2.0.1   
[29] ps_1.6.0          ellipsis_0.3.2    htmltools_0.5.1.1 assertthat_0.2.1 
[33] utf8_1.2.1        stringi_1.7.3     crayon_1.4.1    

In [20]:
hypotheses_table

ERROR: Error in eval(expr, envir, enclos): object 'hypotheses_table' not found


In [13]:
HOG_tibble <- HOG_tibble_complete


# read-in species.tsv - easy lookup of ploidy per species 
species_table <- read.delim("../config/species.tsv", header = TRUE, sep = "\t", row.names = "species")
#species_table
#species_table[expanded_in, "ploidy"]


# via snakemake params - Nmin info for expanded/compared per HOG in hypothesis.tsv (hypothesis specific) are parsed 
# here read-in from table as example
hypotheses_table <- read.delim("../config/hypotheses.tsv", header = TRUE, sep = "\t", row.names = "hypothesis")
Nmin_expanded_in <- hypotheses_table["Nmin_expanded_in",1]
#Nmin_compared_to <- hypotheses_table["Nmin_compared_to",1]
Nmin_compared_to = 1

# expansion factor also in hypotheses.tsv = hypothesis specific
#expansion_factor <- as.numeric(hypotheses_table["min_expansion_factor", 1])
expansion_factor = 2

expansion_difference = 6
ploidy_normalization = "NO"

for (e in expanded_in) {
    # create expansion counter column for each expanded species
    exp_species_name = paste0("exp_counter_", e)
    HOG_tibble <- HOG_tibble %>% tibble::add_column(!!(exp_species_name) := 0)

    if (ploidy_normalization == "YES") {
        for (c in c_t_species) {
            HOG_tibble <- HOG_tibble %>% 
                            mutate(!!(exp_species_name) := 
                            dplyr::case_when(
                                # although cases in which the ct species has no genes are ignored via 
                                # the multiplication in the 2nd step, this underlines that we do so
                                # now with the second expansion distance criterium we should have it anyway
                                get(c) > 0 &
                                get(e)/species_table[e, "ploidy"]*2 >= expansion_factor*(get(c))/species_table[c, "ploidy"]*2 ~ get(exp_species_name) + 1,
                                get(c) > 0 &
                                get(e)/species_table[e, "ploidy"]*2 >= expansion_difference + (get(c))/species_table[c, "ploidy"]*2 ~ get(exp_species_name) + 1,
                                TRUE ~ 0,
                              )
                        )  
        }
    }
    #if the user did not set ploidy_normalization to "YES"
    #here, no ploidy information is used to divide the number of genes per OG
    else {
        for (c in c_t_species) {
            HOG_tibble <- HOG_tibble %>% 
                            mutate(!!(exp_species_name) := 
                            dplyr::case_when(
                                get(c) > 0 &
                                get(e) >= expansion_factor*(get(c)) ~ get(exp_species_name) + 1,
                                get(c) > 0 &
                                get(e) >= expansion_difference + get(c) ~ get(exp_species_name) + 1,
                                TRUE ~ 0,
                              )
                        )
        }
    }
       
}


HOG_tibble %>% 
  arrange(desc(exp_counter_Hordeum_vulgare)) %>% head(n=30)


#print(HOG_tibble)

# then perform summing over all exp_counter_ columns based on user choices
# at least Nmin_expanded_in expanded species that are expanded in at least Nmin_compared_to compared_to species
# create new column
# for each exp_counter_* species check if value >= y
# all passing exp_counter_* species are counted; if sum >= x retain HOG

HOG_tibble <- HOG_tibble %>%
  mutate(pass =
    # selecting columns that we want to use use for subsequent function
    select(., contains("exp_counter_")) %>%
    # use pmap on this subset to get a vector of min from each row
    # dataframe is a list so pmap works on each element of the list; here, each row  
    # we sum the occurences (per row!) of exp_counter_* cols that greater/equal to user cutoff 
    purrr::pmap_dbl(., ~ sum(c(...) >= Nmin_compared_to))
  )


# lastly, retain rows/HOGs that pass x cutoff = number of expanded species
# & drop unnecessary columns
HOG_tibble <- HOG_tibble %>%
  filter(pass >= Nmin_expanded_in) %>%
  select(-contains("exp_counter_"), -pass)


# additional optional filter1 - expanded_in gene family complete criterium
expanded_in_genefam_complete = "yes"
if (expanded_in_genefam_complete == "yes") {
  HOG_tibble <- HOG_tibble %>%
  # create column with per-row sum of expanded_in species that have at least 1 gene in the HOG
  mutate(expanded_in_pass =
    select(., contains(expanded_in)) %>%
    purrr::pmap_dbl(., ~ sum(c(...) >= 1))
  ) %>%
  # only keep rows/HOGs in which at least 1 gene of each expanded_in species occurs
  filter(expanded_in_pass >= length(expanded_in)) %>%
  # remove expanded_in_pass column
  select(-expanded_in_pass)
}


# additional optional filter2 - compared_to gene family complete criterium
compared_to_genefam_complete = "yes"
if (compared_to_genefam_complete == "yes") {
  HOG_tibble <- HOG_tibble %>%
  # create column with per-row sum of compared_to species that have at least 1 gene in the HOG
  mutate(compared_to_pass =
    select(., contains(compared_to)) %>%
    purrr::pmap_dbl(., ~ sum(c(...) >= 1))
  ) %>%
  # only keep rows/HOGs in which at least 1 gene of each compared_to species occurs
  filter(compared_to_pass >= length(compared_to)) %>%
  # remove compared_to_pass column
  select(-compared_to_pass)
}


expanded_HOGs <- HOG_tibble

head(expanded_HOGs, n=10)
nrow(expanded_HOGs)

HOG,Zea_mays,Hordeum_vulgare,Arabidopsis_thaliana,exp_counter_Zea_mays,exp_counter_Hordeum_vulgare
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
N0.HOG0000011,1,64,3,0,1
N0.HOG0000027,0,56,1,0,1
N0.HOG0000037,2,4,2,0,1
N0.HOG0000038,7,16,4,0,1
N0.HOG0000047,0,46,3,0,1
N0.HOG0000049,33,14,1,1,1
N0.HOG0000054,0,45,1,0,1
N0.HOG0000061,1,42,1,0,1
N0.HOG0000075,1,22,11,0,1


HOG,Zea_mays,Hordeum_vulgare,Arabidopsis_thaliana
<chr>,<dbl>,<dbl>,<dbl>
N0.HOG0000011,1,64,3
N0.HOG0000037,2,4,2
N0.HOG0000038,7,16,4
N0.HOG0000049,33,14,1
N0.HOG0000061,1,42,1
N0.HOG0000075,1,22,11
N0.HOG0000076,14,25,3
N0.HOG0000083,7,30,4
N0.HOG0000086,4,31,5


[1] 438

In [22]:
expanded_HOGs[expanded_HOGs$HOG=="N0.HOG0000027",]

HOG,Zea_mays,Hordeum_vulgare,Arabidopsis_thaliana
<chr>,<dbl>,<dbl>,<dbl>


In [14]:
# based on filtering criteria create per hypothesis table with:
# all HOGs and gene counts + additional column expansion (yes/no)
# this is later used in the creation of final tea outputs to create a HOG level table per hypothesis
#HOG_tibble_complete 

#create expansion vector with length of expandsion HOGs
expansion <- replicate(nrow(expanded_HOGs), "yes")

#HOG_tibble_complete  add_column(expanded_HOGs, expansion, .after = "HOG")

#?full_join

expansion_tibble <- full_join(HOG_tibble_complete, add_column(expanded_HOGs, expansion, .after = "HOG"), 
                       by = c("HOG"), suffix = c("", ".remove")) %>% 
                           replace_na(list(expansion = "no")) %>%
                               select(-c(ends_with(".remove"))) 

expansion_tibble

#saveRDS(expansion_tibble, paste("tea/", num, "/expansion_tibble/expansion_tibble.rds", sep = ""))






HOG,Zea_mays,Hordeum_vulgare,Arabidopsis_thaliana,expansion
<chr>,<dbl>,<dbl>,<dbl>,<chr>
N0.HOG0000000,116,0,0,no
N0.HOG0000001,103,0,1,no
N0.HOG0000002,104,0,0,no
N0.HOG0000003,0,100,0,no
N0.HOG0000004,0,93,0,no
N0.HOG0000005,0,81,0,no
N0.HOG0000006,81,0,0,no
N0.HOG0000007,1,79,0,no
N0.HOG0000008,0,75,0,no


In [ ]:
#additional quotation mark as first element is introduced for some reason
#(delim space after : isn't handled properly)
#workaround, we get rid with trailing [-1]
#exp_og_genes <- unlist(strsplit(orthogroups[orthogroups$OG == "OG0005769",]$genes, split = " "))[-1]
#exp_og_genes
#y <- dplyr::filter(all_BLAST_reformatted, qseqid_name %in% exp_og_genes | sseqid_name %in% exp_og_genes)
#sort the results by evalue (ascending V11), bitscore (descending V12) and pident (descending 3)
#w
#w[order(w$evalue, -w$bitscore, -w$pident),]
#y <- arrange(y, evalue, -bitscore, -pident)
#y
#y$qseqid_name
#list_qseqid <- as.character(y$qseqid_name)
#list_sseqid <- as.character(y$sseqid_name)
#list_merged <- unique(c(list_qseqid, list_sseqid))
#list_merged

In [ ]:
#for each gene/protein name in an interesting OG do (is there a file just with names per OG?):
#check "all_BLAST_reformatted" for all entries including these names and create new dataframe/tibble
# then, perform filtering and retain certain set of genes/proteins per OG analysis
#  then, create .txt file per OG with these gene names
## come-up with filter criteria to have better trees?
## I could of course just keep everything and save the evalues, etc.; well, problem for later.. ;D
####> output for snakemake? what about inidividual OG txt files, because starting here parallelisation can really impact

#additional quotation mark as first element is introduced for some reason
#(delim space after : isn't handled properly)
#workaround, we get rid with trailing [-1]

#for (i in expanded_OGs$Orthogroup) {
#    exp_og_genes <- unlist(strsplit(orthogroups[orthogroups$OG == i,]$genes, split = " "))[-1]
#    BLAST_hits_exp_og_genes <- dplyr::filter(all_BLAST_reformatted, 
#                                             qseqid_name %in% exp_og_genes | sseqid_name %in% exp_og_genes)
#    sorted_BLAST_hits_exp_og_genes <- arrange(BLAST_hits_exp_og_genes, evalue, -bitscore, -pident)
#    list_qseqid <- as.character(sorted_BLAST_hits_exp_og_genes$qseqid_name)
#    list_sseqid <- as.character(sorted_BLAST_hits_exp_og_genes$sseqid_name)
#    list_merged <- unique(c(list_qseqid, list_sseqid))
#    write_lines(list_merged,
#           paste("tea/", num, "/exp_OGs_proteinnames/proteinnames_", i, ".txt", sep = ""))
#}

#lastly create .check to know it's done
#exp_OGs_proteinnames.check <- "check"
#write_file(exp_OGs_proteinnames.check, paste("checks/tea/", num, "/exp_OGs_proteinnames.check", sep = ""))

In [15]:
# create genes column in ph_orthogroups file
# row merge? - no unite function, really handy ;D

head(ph_orthogroups %>% unite("genes", all_of(all_species), sep =", ", na.rm = TRUE, remove = TRUE))
ref_ph_orthogroups <- ph_orthogroups %>% unite("genes", all_of(all_species), sep =", ", na.rm = TRUE, remove = TRUE)

HOG,OG,Gene Tree Parent Clade,genes
<chr>,<chr>,<chr>,<chr>
N0.HOG0000000,OG0000000,n0,"Zm00001d002239_P101, Zm00001d002239_P098, Zm00001d002239_P154, Zm00001d002239_P036, Zm00001d002239_P229, Zm00001d002239_P040, Zm00001d002239_P217, Zm00001d002239_P210, Zm00001d002239_P148, Zm00001d002239_P196, Zm00001d002239_P085, Zm00001d002239_P022, Zm00001d002239_P017, Zm00001d002239_P186, Zm00001d002239_P073, Zm00001d002239_P089, Zm00001d002239_P056, Zm00001d002239_P046, Zm00001d002239_P152, Zm00001d002239_P090, Zm00001d002239_P129, Zm00001d002239_P118, Zm00001d002239_P103, Zm00001d002239_P058, Zm00001d002239_P042, Zm00001d002239_P220, Zm00001d002239_P174, Zm00001d002239_P083, Zm00001d002239_P047, Zm00001d002239_P054, Zm00001d002239_P191, Zm00001d002239_P198, Zm00001d002239_P234, Zm00001d002239_P150, Zm00001d002239_P133, Zm00001d002239_P096, Zm00001d002239_P180, Zm00001d002239_P233, Zm00001d002239_P014, Zm00001d002239_P111, Zm00001d002239_P072, Zm00001d002239_P171, Zm00001d002239_P216, Zm00001d002239_P107, Zm00001d002239_P011, Zm00001d002239_P086, Zm00001d002239_P212, Zm00001d002239_P172, Zm00001d002239_P179, Zm00001d002239_P025, Zm00001d002239_P156, Zm00001d002239_P182, Zm00001d002239_P219, Zm00001d002239_P037, Zm00001d002239_P087, Zm00001d002239_P159, Zm00001d002239_P109, Zm00001d002239_P035, Zm00001d002239_P205, Zm00001d002239_P145, Zm00001d002239_P120, Zm00001d002239_P226, Zm00001d002239_P074, Zm00001d002239_P190, Zm00001d002239_P189, Zm00001d002239_P039, Zm00001d002239_P138, Zm00001d002239_P193, Zm00001d002239_P211, Zm00001d002239_P084, Zm00001d002239_P149, Zm00001d002239_P055, Zm00001d002239_P206, Zm00001d002239_P143, Zm00001d002239_P028, Zm00001d002239_P117, Zm00001d002239_P033, Zm00001d002239_P018, Zm00001d002239_P016, Zm00001d002239_P195, Zm00001d002239_P119, Zm00001d002239_P124, Zm00001d002239_P192, Zm00001d002239_P100, Zm00001d002239_P113, Zm00001d002239_P227, Zm00001d002239_P218, Zm00001d002239_P001, Zm00001d002239_P136, Zm00001d002239_P012, Zm00001d002239_P026, Zm00001d002239_P029, Zm00001d002239_P071, Zm00001d002239_P041, Zm00001d002239_P123, Zm00001d002239_P070, Zm00001d002239_P187, Zm00001d002239_P163, Zm00001d002239_P075, Zm00001d002239_P082, Zm00001d002239_P202, Zm00001d002239_P112, Zm00001d002239_P204, Zm00001d002239_P122, Zm00001d002239_P173, Zm00001d002239_P060, Zm00001d002239_P088, Zm00001d002239_P030, Zm00001d002239_P032, Zm00001d002239_P099, Zm00001d002239_P059, Zm00001d002239_P155, Zm00001d002239_P102, Zm00001d002239_P141, Zm00001d002239_P125, Zm00001d002239_P105"
N0.HOG0000001,OG0000001,n0,"Zm00001d006323_P129, Zm00001d006323_P146, Zm00001d006323_P084, Zm00001d006323_P098, Zm00001d006323_P091, Zm00001d006323_P102, Zm00001d006323_P030, Zm00001d006323_P173, Zm00001d006323_P174, Zm00001d006323_P172, Zm00001d006323_P001, Zm00001d006323_P157, Zm00001d006323_P176, Zm00001d006323_P171, Zm00001d006323_P133, Zm00001d006323_P158, Zm00001d006323_P163, Zm00001d006323_P139, Zm00001d006323_P155, Zm00001d006323_P161, Zm00001d006323_P112, Zm00001d006323_P086, Zm00001d006323_P015, Zm00001d006323_P149, Zm00001d006323_P148, Zm00001d006323_P140, Zm00001d006323_P136, Zm00001d006323_P127, Zm00001d006323_P152, Zm00001d006323_P137, Zm00001d006323_P138, Zm00001d006323_P131, Zm00001d006323_P141, Zm00001d006323_P168, Zm00001d006323_P103, Zm00001d006323_P044, Zm00001d006323_P055, Zm00001d006323_P028, Zm00001d006323_P118, Zm00001d006323_P008, Zm00001d006323_P010, Zm00001d006323_P049, Zm00001d006323_P058, Zm00001d006323_P064, Zm00001d006323_P065, Zm00001d006323_P096, Zm00001d006323_P051, Zm00001d006323_P013, Zm00001d006323_P101, Zm00001d006323_P069, Zm00001d006323_P081, Zm00001d006323_P002, Zm00001d006323_P070, Zm00001d006323_P004, Zm00001d006323_P060, Zm00001d006323_P037, Zm00001d006323_P020, Zm00001d006323_P108, Zm00001d006323_P022, Zm00001d006323_P105, Zm00001d006323_P122, Zm00001d006323_P100, Zm00001d006323_P003, Zm00001d006323_P039, Zm00001d006323_P034, Zm00001d006323_P080, Zm00001d006323_P077, Zm00001d006323_P059, Zm00001d006323_P016,

In [ ]:
#exp_og_genes <- unlist(strsplit(ref_ph_orthogroups[ref_ph_orthogroups$HOG == "N0.HOG0000783",]$genes, split = ", "))
#BLAST_hits_exp_og_genes <- dplyr::filter(all_BLAST_reformatted, 
#                                             qseqid_name %in% as.vector(exp_og_genes) | 
#                                             sseqid_name %in% exp_og_genes)# |
#                                              sseqid_name %in% "HORVU1Hr1G015850")
#BLAST_hits_exp_og_genes
#exp_og_genes
#head(all_BLAST_reformatted[all_BLAST_reformatted$sseqid_name=="HORVU1Hr1G015850",])
#BLAST_hits_exp_og_genes
#?filter
#dput(exp_og_genes)

#sorted_BLAST_hits_exp_og_genes <- arrange(BLAST_hits_exp_og_genes, evalue, -bitscore, -pident)
#    list_qseqid <- as.character(sorted_BLAST_hits_exp_og_genes$qseqid_name)
#    list_sseqid <- as.character(sorted_BLAST_hits_exp_og_genes$sseqid_name)
#    list_merged <- unique(c(list_qseqid, list_sseqid))
#list_merged

##### playaround
##### adding a specific number of additional best blast hits to a HOG

In [26]:
exp_og_genes <- unlist(strsplit(ref_ph_orthogroups[ref_ph_orthogroups$HOG == expanded_HOGs$HOG[47],]$genes, 
                                        split = ", "))
#exp_og_genes
BLAST_hits_exp_og_genes <- dplyr::filter(all_BLAST_reformatted, 
                                             qseqid_name %in% exp_og_genes | sseqid_name %in% exp_og_genes)
sorted_BLAST_hits_exp_og_genes <- arrange(BLAST_hits_exp_og_genes, evalue, -bitscore, -pident)
length(exp_og_genes)
#sorted_BLAST_hits_exp_og_genes
add_blast_hits <- 20
HOG_set_extended <- length(exp_og_genes) + add_blast_hits 


# if we have more than "add_blast_hits" further blast hits; limit the set to HOG + "add_blast_hits" param
#if (length(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name)) >= HOG_set_extended) {
    list_qseqid <- as.character(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name)[1:HOG_set_extended])
#}
# if we have less than "add_blast_hits" further blast hits; just take the complete set
#else {
#    list_qseqid <- as.character(sorted_BLAST_hits_exp_og_genes$qseqid_name)
#}

length(list_qseqid)

[1] 24

[1] 44

In [ ]:
#exp_og_genes

12.05.2021  
- ok, so this is actually way less complicated because I can take advantage how vectors work in R
- set1 represents qseqids and set2 represents sseqids
- unique() on both vectors concatenated rowwise! will yield a non-redundant set, maintaing order of appearance in vectors
- if we substract the set of HOG genes (represented by h), then the remaining set is the n next blast hits (in order of filtering)

In [ ]:
set1 <- c("a", "b", "c", "d", "e")
set2 <- c("b", "g", "c", "c", "f")

h <- c("a", "b", "c")

m <- na.omit(unique(c(rbind(set1, set2))))
m
#na.omit(unique(c(rbind(set2, set1))))

#setdiff will remove duplicates - which we shouldn't have here anymore
setdiff(m, h)

In [ ]:
expanded_HOGs$HOG[20]

**retaining up to a specific number of additional BLAST hits + order and info of the BLAST**

In [ ]:
exp_og_genes <- unlist(strsplit(ref_ph_orthogroups[ref_ph_orthogroups$HOG == expanded_HOGs$HOG[20],]$genes, 
                                        split = ", "))

BLAST_hits_exp_og_genes <- dplyr::filter(all_BLAST_reformatted, 
                                             qseqid_name %in% exp_og_genes | sseqid_name %in% exp_og_genes)

sorted_BLAST_hits_exp_og_genes <- arrange(BLAST_hits_exp_og_genes, evalue, -bitscore, -pident)

# sorted table of all HOG genes with their extended blast hits
#sorted_BLAST_hits_exp_og_genes

# set value to extend the BLAST hits
add_blast_hits <- 40
HOG_set_extended <- length(exp_og_genes) + add_blast_hits

# inside of for loop create per HOG extended gene list based on the additional blast hits
# if we have more than "add_blast_hits" further blast hits; limit the set to HOG + "add_blast_hits" param
#if (length(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name)) >= HOG_set_extended) {
#   list_qseqid <- as.character(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name)[1:HOG_set_extended])
#}
# if we have less than "add_blast_hits" further blast hits; just take the complete set
#else {
#    list_qseqid <- as.character(sorted_BLAST_hits_exp_og_genes$qseqid_name)
#}

#list_qseqid <- as.character(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name)[1:HOG_set_extended])
#list_sseqid <- as.character(unique(sorted_BLAST_hits_exp_og_genes$sseqid_name)[1:HOG_set_extended])
#list_merged <- na.omit(unique(c(list_qseqid, list_sseqid)))
                            
# ...
# final object is called "list_merged"
#list_qseqid
#list_sseqid
#list_merged

# use tail() to access last element in vector
#tail(list_merged, n=1)

# get row number of last appearance of last element in ssequid column
# in some instances not as straightforward and last hit is actually on the query side
# then use last entry rownumber there as subset cutoff
#last <- sorted_BLAST_hits_exp_og_genes %>%
#  rowid_to_column() %>%
#  filter(
#    case_when(
#      sseqid_name != tail(list_merged, n=1) ~ qseqid_name == tail(list_merged, n=1),
#      T ~ sseqid_name == tail(list_merged, n=1)
##    )
#  ) %>%
#  tail(n=1)
#last


# get gene name of last gene to be added based on number of add_blast_hits
all_blast_genes <- na.omit(
       unique(
         c(
           rbind(
             sorted_BLAST_hits_exp_og_genes$qseqid_name, 
             sorted_BLAST_hits_exp_og_genes$sseqid_name
           )
         )
       )
     ) 

#length(all_blast_genes)


#length(unique(exp_og_genes))
#length(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name))

# set of all extended blast hits (based on threshold) - vector of gene names (ordered!)
# also nice: don't need a conditional since `%>% head(n = add_blast_hits)` will work,
# even if add_blast_hits param is > setdiff(all_blast_genes, exp_og_genes) 
extended_blast_hits_genes <- setdiff(all_blast_genes, exp_og_genes) %>% head(n = add_blast_hits)
#length(extended_blast_hits_genes)

# non redundant set of gene names of HOG + n additional blast hits as defined in the user threshold
HOG_and_ext_blast_hits_genes <- c(exp_og_genes, extended_blast_hits_genes)
length(HOG_and_ext_blast_hits_genes)

#create subset of sorted_BLAST_hits_exp_og_genes table in which only:
# exp_og_genes & extended_blast_hits_genes are allowed to appear
# this way we have cutoff for the nth best blast hit/gene but also keep all secondary hits
HOG_and_ext_blast_hits_table <- sorted_BLAST_hits_exp_og_genes %>%
                                        filter(qseqid_name %in% HOG_and_ext_blast_hits_genes) %>%
                                        filter(sseqid_name %in% HOG_and_ext_blast_hits_genes)

head(HOG_and_ext_blast_hits_table)

# create a subset of the "sorted_BLAST_hits_exp_og_genes" dataframe based on the final extended gene list
# take last gene of extended set, search sseqid until last appearance and cut dataframe there?..
# just keep all vs all information - additonal filtering as part of the WebApp
#subset_sorted_BLAST_hits_exp_og_genes <- sorted_BLAST_hits_exp_og_genes[1:last$rowid,]
#subset_sorted_BLAST_hits_exp_og_genes



# finally create object per HOG which stores the BLAST information
# object@name_of_HOG <- prev. subsetted tibble of BLAST hits

In [ ]:
## define custom class
# need a list object to hold all data of this class
extended_BLAST_hits <- list()

# class for extended BLAST hits info
setClass("extended_BLAST_hits", 
         slots=list(blast_table="tbl_df",
                    nrow_table="numeric",
                    num_genes_HOG="numeric",
                    num_genes_extend="numeric",
                    num_genes_complete="numeric",
                    genes_HOG="tbl_df")#,
#                    genes_extend="tbl_df",
#                    genes_complete="tbl_df")
         )

ext_B_hits <- new("extended_BLAST_hits",
      blast_table=subset_sorted_BLAST_hits_exp_og_genes,
      nrow_table=nrow(subset_sorted_BLAST_hits_exp_og_genes),
      num_genes_HOG=length(exp_og_genes),
      num_genes_extend = length(
                  unique(
                    c(
                      unique(subset_sorted_BLAST_hits_exp_og_genes$qseqid_name), 
                      unique(subset_sorted_BLAST_hits_exp_og_genes$sseqid_name)
                      )
                  )
                ) - length(exp_og_genes),
      num_genes_complete=length(
                  unique(
                    c(
                      unique(subset_sorted_BLAST_hits_exp_og_genes$qseqid_name), 
                      unique(subset_sorted_BLAST_hits_exp_og_genes$sseqid_name)
                      )
                  )
                ),
      genes_HOG=as_tibble(exp_og_genes)#,
#      genes_extend=as_tibble(unique(
#                    c(
#                      unique(subset_sorted_BLAST_hits_exp_og_genes$qseqid_name), 
#                      unique(subset_sorted_BLAST_hits_exp_og_genes$sseqid_name)
#                      )) - exp_og_genes),
#      genes_complete=as_tibble(unique(
#                    c(
#                      unique(subset_sorted_BLAST_hits_exp_og_genes$qseqid_name), 
#                      unique(subset_sorted_BLAST_hits_exp_og_genes$sseqid_name)
#                      )))
    )

ext_B_hits <- list(ext_B_hits)
names(ext_B_hits) <- paste0(expanded_HOGs$HOG[21])

#x <- list(test)
#names(x) <- paste0(expanded_HOGs$HOG[47])

extended_BLAST_hits <- c(extended_BLAST_hits, ext_B_hits)

summary(extended_BLAST_hits)
str(extended_BLAST_hits)

#ext_B_hits$N0.HOG0000250

In [ ]:
extended_BLAST_hits$N0.HOG0000101@genes_HOG

In [ ]:
#now HOG based

#for each gene/protein name in an interesting OG do (is there a file just with names per OG?):
#check "all_BLAST_reformatted" for all entries including these names and create new dataframe/tibble
# then, perform filtering and retain certain set of genes/proteins per OG analysis
#  then, create .txt file per OG with these gene names
## come-up with filter criteria to have better trees?
## I could of course just keep everything and save the evalues, etc.; well, problem for later.. ;D
####> output for snakemake? what about inidividual OG txt files, because starting here parallelisation can really impact

#additional quotation mark as first element is introduced for some reason
#(delim space after : isn't handled properly)
#workaround, we get rid with trailing [-1]
#not a problem anymore with N0.tsv table as input

dir.create(paste("tea/", num, "/extended_BLAST_hits/", sep = ""))

# define the number of additional best blast hits to include in the follow-up analyses
add_blast_hits <- 20

## define custom class for extended blast hits
# need a list object to hold all data of this class
extended_BLAST_hits <- list()

# class for extended BLAST hits info
setClass("extended_BLAST_hits", 
         slots=list(blast_table="tbl_df",
                    num_genes_HOG="numeric",
                    num_genes_extend="numeric",
                    num_genes_complete="numeric",
                    genes_HOG="tbl_df",
                    genes_extend_hits="tbl_df")
         )


for (i in expanded_HOGs$HOG) {
    # split = ", " (instead of " ") is super important; didn't see this bug at first but
    # it led to most genes not being recognized due to additional "," at the end..! (see tests in prev. cell)
    exp_og_genes <- unlist(strsplit(ref_ph_orthogroups[ref_ph_orthogroups$HOG == i,]$genes, split = ", "))
    BLAST_hits_exp_og_genes <- dplyr::filter(all_BLAST_reformatted, 
                                             qseqid_name %in% exp_og_genes | sseqid_name %in% exp_og_genes)
    sorted_BLAST_hits_exp_og_genes <- arrange(BLAST_hits_exp_og_genes, evalue, -bitscore, -pident)
    # add number of chosen additional best blast hits to size of HOG 
    HOG_set_extended <- length(exp_og_genes) + add_blast_hits
    
    ### first qseqid
    ### if we have more than "additional genes" further blast hits; limit the set to HOG + "additional genes" param
    #if (length(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name)) >= HOG_set_extended) {
    ##    list_qseqid <- as.character(unique(sorted_BLAST_hits_exp_og_genes$qseqid_name)[1:HOG_set_extended])
    ##}
    ## if we have less than "additional genes" further blast hits; just take the complete set
    ##else {
    ##    list_qseqid <- as.character(sorted_BLAST_hits_exp_og_genes$qseqid_name)
    ##}
    ### now for sseqid
    ## if we have more than "additional genes" further blast hits; limit the set to HOG + "additional genes" param
    ##if (length(unique(sorted_BLAST_hits_exp_og_genes$sseqid_name)) >= HOG_set_extended) {
    ##    list_sseqid <- as.character(unique(sorted_BLAST_hits_exp_og_genes$sseqid_name)[1:HOG_set_extended])
    ##}
    ## if we have less than "additional genes" further blast hits; just take the complete set
    ##else {
    ##    list_sseqid <- as.character(sorted_BLAST_hits_exp_og_genes$sseqid_name)
    ##}
    ##list_merged <- unique(c(list_qseqid, list_sseqid))
    
    # get gene name of last gene to be added based on number of add_blast_hits
    all_blast_genes <- na.omit(
       unique(
         c(
           rbind(
             sorted_BLAST_hits_exp_og_genes$qseqid_name, 
             sorted_BLAST_hits_exp_og_genes$sseqid_name
           )
         )
       )
     ) 
    
    # set of all extended blast hits (based on threshold) - vector of gene names (ordered!)
    # also nice: don't need a conditional since `%>% head(n = add_blast_hits)` will work,
    # even if add_blast_hits param is > setdiff(all_blast_genes, exp_og_genes) 
    extended_blast_hits_genes <- setdiff(all_blast_genes, exp_og_genes) %>% head(n = add_blast_hits)
    # length(extended_blast_hits_genes)
  
    # non redundant set of gene names of HOG + n additional blast hits as defined in the user threshold
    HOG_and_ext_blast_hits_genes <- c(exp_og_genes, extended_blast_hits_genes)

    #create subset of sorted_BLAST_hits_exp_og_genes table in which only:
    # exp_og_genes & extended_blast_hits_genes are allowed to appear
    # this way we have cutoff for the nth best blast hit/gene but also keep all secondary hits
    HOG_and_ext_blast_hits_table <- sorted_BLAST_hits_exp_og_genes %>%
                                      filter(qseqid_name %in% HOG_and_ext_blast_hits_genes) %>%
                                      filter(sseqid_name %in% HOG_and_ext_blast_hits_genes)
    
    write_lines(HOG_and_ext_blast_hits_genes,
           paste("tea/", num, "/exp_OGs_proteinnames/proteinnames_", i, ".txt", sep = ""))
    
    
    # also: create list of S4 object containing the blast hits and respective information
    
    ## use tail() to access last element in vector
    ##tail(list_merged, n=1)

    ## get row number of last appearance of last element in ssequid column
    ## in some instances not as straightforward and last hit is actually on the query side
    ## then use last entry rownumber there as subset cutoff
    ##last <- sorted_BLAST_hits_exp_og_genes %>%
    ##  rowid_to_column() %>%
    ##  filter(
    ##   case_when(
    ##      sseqid_name != tail(list_merged, n=1) ~ qseqid_name == tail(list_merged, n=1),
    ##      T ~ sseqid_name == tail(list_merged, n=1)
    ##    )
    ##  ) %>%
    ##  tail(n=1)
    
    ## create a subset of the "sorted_BLAST_hits_exp_og_genes" dataframe based on the final extended gene list
    ## take last gene of extended set, search sseqid until last appearance and cut dataframe there
    ## just keep all vs all information - additonal filtering as part of the WebApp
    ##subset_sorted_BLAST_hits_exp_og_genes <- sorted_BLAST_hits_exp_og_genes[1:last$rowid,]
    
    # for each exp. HOG create an extended_BLAST_hits S4 object and collect as part of list
    ext_B_hits <- new("extended_BLAST_hits",
      blast_table=HOG_and_ext_blast_hits_table,
    #  nrow_table=nrow(subset_sorted_BLAST_hits_exp_og_genes),
      num_genes_HOG=length(exp_og_genes),
      num_genes_extend = length(extended_blast_hits_genes),
      #            unique(
      #              c(
      #                unique(subset_sorted_BLAST_hits_exp_og_genes$qseqid_name), 
      #                unique(subset_sorted_BLAST_hits_exp_og_genes$sseqid_name)
      #                )
      #            )
      #          ) - length(exp_og_genes),
      num_genes_complete=length(HOG_and_ext_blast_hits_genes),
      #            unique(
      #              c(
      #                unique(subset_sorted_BLAST_hits_exp_og_genes$qseqid_name), 
      #                unique(subset_sorted_BLAST_hits_exp_og_genes$sseqid_name)
      #                )
      #            )
      #          ),
      genes_HOG=as_tibble(exp_og_genes),
      genes_extend_hits=as_tibble(extended_blast_hits_genes)
            )
    # assign name based on name of the underlying expanded HOG
    ext_B_hits <- list(ext_B_hits)
    names(ext_B_hits) <- paste0(i)
    
    # append to list object
    extended_BLAST_hits <- c(extended_BLAST_hits, ext_B_hits)
}

# save extended BLAST hits to hypothesis specific ("num") RDS file 
#-> to be read and used in final_tea_computation.R script
saveRDS(extended_BLAST_hits, paste("tea/", num, "/extended_BLAST_hits/extended_BLAST_hits.RDS", sep = ""))

#lastly create .check to know it's done
exp_OGs_proteinnames.check <- "check"
write_file(exp_OGs_proteinnames.check, paste("checks/tea/", num, "/exp_OGs_proteinnames.check", sep = ""))

In [ ]:
as_tibble(extended_blast_hits_genes)

In [ ]:
extended_BLAST_hits$N0.HOG0000101
length(extended_BLAST_hits)
str(expanded_HOGs)

In [ ]:
paste("checks/tea/", num, "/exp_OGs_proteinnames.check", sep = "")

In [ ]:
exp_OGs_proteinnames.check <- "check"
write_file(exp_OGs_proteinnames.check, paste("checks/tea/", num, "/exp_OGs_proteinnames.check", sep = ""))

In [ ]:
#str(all_BLAST_reformatted)
#?read_delim
library(stringr)
conversion <- conversion %>%  mutate(seqid = str_remove_all(seqid, ":"))
str(conversion)

In [ ]:
#perhaps continue as dataframe; have to see how easy this is

In [ ]:

#anyways after that snakemake steps:
#1)faSomeRecords - extract the fasta sequences of all proteins per interesting OG
#2)muscle - multiple sequence alignment
#3)gblocks - eliminates poorly aligned positions and divergent regions of DNA or protein alignments
#4)fastTree - build trees

#I need to play around a bit with the string to column matching
#there are several solutions to this - see https://stackoverflow.com/questions/25923392/select-columns-based-on-string-match-dplyrselect

### 05.04.2022 - Adding HOGs instead of BLAST hits

This is done in the analysis anyways ;D

In [16]:
exp_og_genes <- unlist(strsplit(ref_ph_orthogroups[ref_ph_orthogroups$HOG == expanded_HOGs$HOG[47],]$genes, 
                                        split = ", "))
head(exp_og_genes)
#exp_og_genes
BLAST_hits_exp_og_genes <- dplyr::filter(all_BLAST_reformatted, 
                                             qseqid_name %in% exp_og_genes | sseqid_name %in% exp_og_genes)
sorted_BLAST_hits_exp_og_genes <- arrange(BLAST_hits_exp_og_genes, evalue, -bitscore, -pident)
#length(exp_og_genes)

[1] "Zm00001d002828_P001" "HORVU2Hr1G092030.19" "HORVU2Hr1G092030.20"
[4] "HORVU2Hr1G092030.18" "HORVU2Hr1G092030.17" "HORVU2Hr1G092030.4"

The ordering is useful for this new approach also! 

In [17]:
#get the HOG in question
ref_ph_orthogroups[ref_ph_orthogroups$HOG == expanded_HOGs$HOG[47],]$HOG

[1] "N0.HOG0000344"

In [18]:
head(sorted_BLAST_hits_exp_og_genes)
names(head(sorted_BLAST_hits_exp_og_genes))
head(sorted_BLAST_hits_exp_og_genes$sseqid_name)

qseqid,qseqid_name,sseqid,sseqid_name,pident,length,mismatch,gapopen,qstart,qend,⋯,X3.y,X4.y,X5.y,X6.y,X7.y,X8.y,X9.y,X10.y,X11.y,X12.y
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
0_6693,AT2G18060.3,0_6693,AT2G18060.3,100,365,0,0,1,365,⋯,pep,chromosome:TAIR10:2:7848320:7850303:-1,gene:AT2G18060,transcript:AT2G18060.3,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:NAC037,description:VND1,[Source:UniProtKB/TrEMBL;Acc:A0A178VM15],NA
0_6693,AT2G18060.3,0_6694,AT2G18060.1,100,365,0,0,1,365,⋯,pep,chromosome:TAIR10:2:7847967:7851553:-1,gene:AT2G18060,transcript:AT2G18060.1,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:NAC037,description:VND1,[Source:UniProtKB/TrEMBL;Acc:A0A178VM15],NA
0_6694,AT2G18060.1,0_6693,AT2G18060.3,100,365,0,0,1,365,⋯,pep,chromosome:TAIR10:2:7848320:7850303:-1,gene:AT2G18060,transcript:AT2G18060.3,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:NAC037,description:VND1,[Source:UniProtKB/TrEMBL;Acc:A0A178VM15],NA
0_6694,AT2G18060.1,0_6694,AT2G18060.1,100,365,0,0,1,365,⋯,pep,chromosome:TAIR10:2:7847967:7851553:-1,gene:AT2G18060,transcript:AT2G18060.1,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:NAC037,description:VND1,[Source:UniProtKB/TrEMBL;Acc:A0A178VM15],NA
2_2994,Zm00001d002828_P001,2_2994,Zm00001d002828_P001,100,344,0,0,1,344,⋯,pep,chromosome:B73_RefGen_v4:2:24002536:24006073:1,gene:Zm00001d002828,transcript:Zm00001d002828_T001,gene_biotype:protein_coding,transcript_biotype:protein_coding,gene_symbol:NAC-transcription,factor,131,description:NAC domain-containing protein 26
1_33053,HORVU2Hr1G092030.4,1_33053,HORVU2Hr1G092030.4,100,339,0,0,1,339,⋯,pep,chromosome:IBSC_v2:chr2H:650452538:650457232:-1,gene:HORVU2Hr1G092030,transcript:HORVU2Hr1G092030.4,gene_biotype:protein_coding,transcript_biotype:protein_coding,NA,NA,NA,NA


[1] "qseqid"      "qseqid_name" "sseqid"      "sseqid_name" "pident"     
 [6] "length"      "mismatch"    "gapopen"     "qstart"      "qend"       
[11] "sstart"      "send"        "evalue"      "bitscore"    "X3.x"       
[16] "X4.x"        "X5.x"        "X6.x"        "X7.x"        "X8.x"       
[21] "X9.x"        "X10.x"       "X11.x"       "X12.x"       "X3.y"       
[26] "X4.y"        "X5.y"        "X6.y"        "X7.y"        "X8.y"       
[31] "X9.y"        "X10.y"       "X11.y"       "X12.y"

[1] "AT2G18060.3"         "AT2G18060.1"         "AT2G18060.3"        
[4] "AT2G18060.1"         "Zm00001d002828_P001" "HORVU2Hr1G092030.4"

In [19]:
#we can shorten this table already
short_sorted_BLAST_hits_exp_og_genes <- sorted_BLAST_hits_exp_og_genes %>%
  select(sseqid_name, evalue, bitscore, pident)

head(short_sorted_BLAST_hits_exp_og_genes)

sseqid_name,evalue,bitscore,pident
<chr>,<dbl>,<dbl>,<dbl>
AT2G18060.3,4.19e-279,752,100
AT2G18060.1,4.19e-279,752,100
AT2G18060.3,4.19e-279,752,100
AT2G18060.1,4.19e-279,752,100
Zm00001d002828_P001,4.05e-270,729,100
HORVU2Hr1G092030.4,5.91e-257,696,100


In [20]:
#transforming ph_orthogroups to long format
long_ph_orthogroups <- ph_orthogroups %>%
  select(-OG, -`Gene Tree Parent Clade`) %>%
  melt(., id.vars = c("HOG")) %>%
  rename(species=variable, id=value) %>%
  mutate(species = as.character(species)) %>%
  separate_rows(id, sep = ", ") %>%
  drop_na()

head(long_ph_orthogroups)

HOG,species,id
<chr>,<chr>,<chr>
N0.HOG0000001,Arabidopsis_thaliana,AT2G31650.1
N0.HOG0000011,Arabidopsis_thaliana,AT2G11035.1
N0.HOG0000011,Arabidopsis_thaliana,AT2G02520.1
N0.HOG0000011,Arabidopsis_thaliana,AT2G22440.1
N0.HOG0000017,Arabidopsis_thaliana,AT2G28540.2
N0.HOG0000017,Arabidopsis_thaliana,AT2G28540.1


In [21]:
#we can create a merged dataframe from the blast table (HOG specific) and the long format HOG table (general)
#difficulty is that in case of singletons we have NA under HOG
#easiest way I can think of is a named list
self_and_closest_ogs <- left_join(short_sorted_BLAST_hits_exp_og_genes, long_ph_orthogroups, by = c("sseqid_name" = "id")) %>%
  #and follow this up by reducing it to the HOG column
  #pull(HOG) %>%
  #remove duplicates
  #unique() %>%
#  filter(is.na(species)) %>%
  group_by(sseqid_name) %>%
  arrange(evalue, -bitscore, -pident) %>%
  slice(1) %>%
  ungroup() %>%
  arrange(evalue, -bitscore, -pident) %>%
  mutate(HOG = as.character(HOG),
         HOG = ifelse(is.na(HOG), paste("singleton", sep = "-", cumsum(is.na(HOG))),
                      as.character(HOG))) %>%
  group_by(HOG) %>%
  arrange(evalue, -bitscore, -pident) %>%
  slice(1) %>%
  ungroup() %>%
  arrange(evalue, -bitscore, -pident)
 

#first row will should correspond to self match - the OG itself
self_and_closest_ogs


#suppose user chose max 5 additional orthogroups in config file
add_OGs = 3


#need to add check for number of additonal OGs/singletons
#e.g. user sets max. add. OGs/singletons to 5 but we only can provide 4
#+ have to consider the expanded OG itself, so:
available_add_OGs <- nrow(self_and_closest_ogs) - 1 
available_add_OGs

#in this case set add_OGs to max available
if (available_add_OGs < add_OGs) {
  add_OGs <- available_add_OGs
} 


#empty list of precomputed size - add OGs plus the expanded OG itself
add_OG_analysis_list <- vector(mode = "list", length = add_OGs+1)


for (i in 1:(add_OGs+1)) {
  og_name <- self_and_closest_ogs[i,] %>%
    pull(HOG)
  
  #differnetiate between OG case and a singleton case - different handling:
  #for HOG get all associated genes/proteins from large long format table
  #for singletons get singelton gene/protein from the table itself
  if (!str_detect(og_name, "singleton") == TRUE) {

    name_curr_close_HOG <- self_and_closest_ogs[i,] %>%
                             pull(HOG)

    ids_curr_close_HOG <- long_ph_orthogroups %>%
                            filter(HOG %in% name_curr_close_HOG) %>%
                            pull(id)

    add_OG_analysis_list[[i]] <- ids_curr_close_HOG
    names(add_OG_analysis_list)[i] <- name_curr_close_HOG

  } else {
    name_curr_close_singleton <- self_and_closest_ogs[i,] %>%
                                   pull(HOG)

    id_curr_close_HOG <- self_and_closest_ogs[i,] %>%
                           pull(sseqid_name)

    add_OG_analysis_list[[i]] <- id_curr_close_HOG
    names(add_OG_analysis_list)[i] <- name_curr_close_singleton
  }
}


str(add_OG_analysis_list)

sseqid_name,evalue,bitscore,pident,HOG,species
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
AT2G18060.1,4.19e-279,752.0,100.0,N0.HOG0000344,Arabidopsis_thaliana
AT2G46770.1,1.53e-87,266.0,45.8,N0.HOG0002992,Arabidopsis_thaliana
Zm00001d002934_P001,3.19e-82,253.0,64.6,N0.HOG0000190,Zea_mays
Zm00001d006053_P001,4.98e-62,202.0,56.7,N0.HOG0001499,Zea_mays
Zm00001d003052_P001,2.56e-54,179.0,52.5,N0.HOG0002300,Zea_mays
AT2G43000.1,7.26e-53,171.0,47.9,N0.HOG0002155,Arabidopsis_thaliana
AT2G33480.2,1.02e-52,173.0,50.6,N0.HOG0003022,Arabidopsis_thaliana
Zm00001d002945_P012,1.96e-51,169.0,48.1,N0.HOG0001862,Zea_mays
Zm00001d002285_P001,1.58e-50,170.0,45.7,N0.HOG0001631,Zea_mays


[1] 16

List of 4
 $ N0.HOG0000344: chr [1:24] "AT2G18060.3" "AT2G18060.1" "AT2G18060.2" "HORVU2Hr1G092030.19" ...
 $ N0.HOG0002992: chr [1:5] "AT2G46770.1" "HORVU2Hr1G077320.3" "HORVU2Hr1G077320.1" "HORVU2Hr1G077320.2" ...
 $ N0.HOG0000190: chr [1:30] "HORVU2Hr1G126080.3" "HORVU2Hr1G126080.4" "HORVU2Hr1G126080.2" "HORVU2Hr1G126080.1" ...
 $ N0.HOG0001499: chr [1:10] "AT2G24430.2" "AT2G24430.1" "HORVU2Hr1G080460.21" "HORVU2Hr1G080460.20" ...


In [22]:
#create copy of the list and remove the names
#add all previous gene/proteins to the next OG
# -> e.g. the former seventh OG list element will contain all genes/proteins of all lower numbered OGs
cum_add_OG_analysis_list <- add_OG_analysis_list
names(cum_add_OG_analysis_list) <- NULL
#copy_add_OG_analysis_list

for (i in 0:(length(cum_add_OG_analysis_list)-1)) {
    
    cum_add_OG_analysis_list[[i+1]] <- unlist(c(cum_add_OG_analysis_list[i], cum_add_OG_analysis_list[i+1]))
}

#lapply(cum_add_OG_analysis_list, length)
cum_add_OG_analysis_list

[[1]]
 [1] "AT2G18060.3"         "AT2G18060.1"         "AT2G18060.2"        
 [4] "HORVU2Hr1G092030.19" "HORVU2Hr1G092030.20" "HORVU2Hr1G092030.18"
 [7] "HORVU2Hr1G092030.17" "HORVU2Hr1G092030.4"  "HORVU2Hr1G092030.5" 
[10] "HORVU2Hr1G092030.11" "HORVU2Hr1G092030.9"  "HORVU2Hr1G092030.2" 
[13] "HORVU2Hr1G092030.3"  "HORVU2Hr1G092030.14" "HORVU2Hr1G092030.1" 
[16] "HORVU2Hr1G092030.6"  "HORVU2Hr1G092030.7"  "HORVU2Hr1G092030.13"
[19] "HORVU2Hr1G092030.15" "HORVU2Hr1G092030.10" "HORVU2Hr1G092030.16"
[22] "HORVU2Hr1G092030.8"  "HORVU2Hr1G092030.12" "Zm00001d002828_P001"

[[2]]
 [1] "AT2G18060.3"         "AT2G18060.1"         "AT2G18060.2"        
 [4] "HORVU2Hr1G092030.19" "HORVU2Hr1G092030.20" "HORVU2Hr1G092030.18"
 [7] "HORVU2Hr1G092030.17" "HORVU2Hr1G092030.4"  "HORVU2Hr1G092030.5" 
[10] "HORVU2Hr1G092030.11" "HORVU2Hr1G092030.9"  "HORVU2Hr1G092030.2" 
[13] "HORVU2Hr1G092030.3"  "HORVU2Hr1G092030.14" "HORVU2Hr1G092030.1" 
[16] "HORVU2Hr1G092030.6"  "HORVU2Hr1G092030.7"  "HORVU2Hr1G092030.13"
[19] "HORVU2Hr1G092030.15" "HORVU2Hr1G092030.10" "HORVU2Hr1G092030.16"
[22] "HORVU2Hr1G092030.8"  "HORVU2Hr1G092030.12" "Zm00001d002828_P001"
[25] "AT2G46770.1"         "HORVU2Hr1G077320.3"  "HORVU2Hr1G077320.1" 
[28] "HORVU2Hr1G077320.2"  "Zm00001d003583_P001"

[[3]]
 [1] "AT2G18060.3"         "AT2G18060.1"         "AT2G18060.2"        
 [4] "HORVU2Hr1G092030.19" "HORVU2Hr1G092030.20" "HORVU2Hr1G092030.18"
 [7] "HORVU2Hr1G092030.17" "HORVU2Hr1G092030.4"  "HORVU2Hr1G092030.5" 
[10] "HORVU2Hr1G092030.11" "HORVU2Hr1G092030.9"  "HORVU2Hr1G092030.2" 
[13] "HORVU2Hr1G092030.3"  "HORVU2Hr1G092030.14" "HORVU2Hr1G092030.1" 
[16] "HORVU2Hr1G092030.6"  "HORVU2Hr1G092030.7"  "HORVU2Hr1G092030.13"
[19] "HORVU2Hr1G092030.15" "HORVU2Hr1G092030.10" "HORVU2Hr1G092030.16"
[22] "HORVU2Hr1G092030.8"  "HORVU2Hr1G092030.12" "Zm00001d002828_P001"
[25] "AT2G46770.1"         "HORVU2Hr1G077320.3"  "HORVU2Hr1G077320.1" 
[28] "HORVU2Hr1G077320.2"  "Zm00001d003583_P001" "HORVU2Hr1G126080.3" 
[31] "HORVU2Hr1G126080.4"  "HORVU2Hr1G126080.2"  "HORVU2Hr1G126080.1" 
[34] "HORVU2Hr1G126080.8"  "HORVU2Hr1G126080.6"  "HORVU2Hr1G126080.7" 
[37] "HORVU2Hr1G126080.9"  "HORVU2Hr1G126080.11" "HORVU2Hr1G126040.3" 
[40] "HORVU2Hr1G126040.14" "HORVU2Hr1G126040.1"  "HORVU2Hr1G126040.13"
[43] "HORVU2Hr1G126040.6"  "HORVU2Hr1G126040.4"  "HORVU2Hr1G126040.2" 
[46] "HORVU2Hr1G126040.11" "HORVU2Hr1G126080.10" "HORVU2Hr1G126080.5" 
[49] "HORVU2Hr1G126040.8"  "HORVU2Hr1G126040.7"  "HORVU2Hr1G126040.10"
[52] "HORVU2Hr1G126040.5"  "HORVU2Hr1G126040.15" "HORVU2Hr1G126040.12"
[55] "HORVU2Hr1G126040.9"  "HORVU2Hr1G126040.16" "HORVU2Hr1G126040.18"
[58] "HORVU2Hr1G126040.17" "Zm00001d002934_P001"

[[4]]
 [1] "AT2G18060.3"         "AT2G18060.1"         "AT2G18060.2"        
 [4] "HORVU2Hr1G092030.19" "HORVU2Hr1G092030.20" "HORVU2Hr1G092030.18"
 [7] "HORVU2Hr1G092030.17" "HORVU2Hr1G092030.4"  "HORVU2Hr1G092030.5" 
[10] "HORVU2Hr1G092030.11" "HORVU2Hr1G092030.9"  "HORVU2Hr1G092030.2" 
[13] "HORVU2Hr1G092030.3"  "HORVU2Hr1G092030.14" "HORVU2Hr1G092030.1" 
[16] "HORVU2Hr1G092030.6"  "HORVU2Hr1G092030.7"  "HORVU2Hr1G092030.13"
[19] "HORVU2Hr1G092030.15" "HORVU2Hr1G092030.10" "HORVU2Hr1G092030.16"
[22] "HORVU2Hr1G092030.8"  "HORVU2Hr1G092030.12" "Zm00001d002828_P001"
[25] "AT2G46770.1"         "HORVU2Hr1G077320.3"  "HORVU2Hr1G077320.1" 
[28] "HORVU2Hr1G077320.2"  "Zm00001d003583_P001" "HORVU2Hr1G126080.3" 
[31] "HORVU2Hr1G126080.4"  "HORVU2Hr1G126080.2"  "HORVU2Hr1G126080.1" 
[34] "HORVU2Hr1G126080.8"  "HORVU2Hr1G126080.6"  "HORVU2Hr1G126080.7" 
[37] "HORVU2Hr1G126080.9"  "HORVU2Hr1G126080.11" "HORVU2Hr1G126040.3" 
[40] "HORVU2Hr1G126040.14" "HORVU2Hr1G126040.1"  "HORVU2Hr1G126040.13"
[43] "HORVU2Hr1G126040.6"  "HORVU2Hr1G126040.4"  "HORVU2Hr1G126040.2" 
[46] "HORVU2Hr1G126040.11" "HORVU2Hr1G126080.10" "HORVU2Hr1G126080.5" 
[49] "HORVU2Hr1G126040.8"  "HORVU2Hr1G126040.7"  "HORVU2Hr1G126040.10"
[52] "HORVU2Hr1G126040.5"  "HORVU2Hr1G126040.15" "HORVU2Hr1G126040.12"
[55] "HORVU2Hr1G126040.9"  "HORVU2Hr1G126

In [319]:
#do this for all expanded OGs
#do this for all hypotheses
#what kind of additional table is worthwhile here? - do I even need one - it's the same as in the additional blast hits stuff ;D
#how do I split this in the following steps muscle - gblocks - fasttree;
# -> prob. a nested structure of directories - I know the number of additional OGs per OG
# -> so OG/; 1/ 2/ 3/, etc.; where 1/ corresponds to the expanded OG itself and the following to the OG itself plus the addtional OG and all prev. add. OGs
#adding this to the final tea object

#### Putting the additional OG analysis together

In [66]:
#transforming ph_orthogroups to long format - nice & neat lookup table ;D
long_ph_orthogroups <- ph_orthogroups %>%
  select(-OG, -`Gene Tree Parent Clade`) %>%
  melt(., id.vars = c("HOG")) %>%
  rename(species=variable, id=value) %>%
  mutate(species = as.character(species)) %>%
  separate_rows(id, sep = ", ") %>%
  drop_na()


#create final summary list - per OG (name) the cumulative steps of additional OGs
summary_add_OG_analysis_list <- vector(mode = "list", length = length(expanded_HOGs$HOG))

#create classes for nested S3-list structure holding all additional OG sets per OG analysis
setClass("add_OG_analysis", 
         slots=list(add_OG_analysis="list")
         )

setClass("add_OG_set", 
         slots=list(genes="tbl_df")
         )


num=1
#dir.create(paste("../tea/", num, "/add_OGs_sets/", sep = ""))
#dir.create(paste("tea/", num, "/add_OGs_sets/", sep = ""))

#similarly to the additional BLAST hits approach we iterate over the expanded OGs
loop_index <- 0
for (i in expanded_HOGs$HOG) {
    
    #not pretty but was quick solution
    loop_index <- loop_index + 1
    
    exp_og_genes <- unlist(strsplit(ref_ph_orthogroups[ref_ph_orthogroups$HOG == i,]$genes, split = ", "))
    BLAST_hits_exp_og_genes <- dplyr::filter(all_BLAST_reformatted, 
                                             qseqid_name %in% exp_og_genes | sseqid_name %in% exp_og_genes)
    sorted_BLAST_hits_exp_og_genes <- arrange(BLAST_hits_exp_og_genes, evalue, -bitscore, -pident)

    #after the sorting of BLAST hits we move on to merging the OG information into the current exp. OG BLAST hits results

    #we can create a merged dataframe from the blast table (HOG specific) and the long format HOG table (general)
    #difficulty is that in case of singletons we have NA under HOG
    #easiest way I can think of is a named list
    self_and_closest_ogs <- left_join(sorted_BLAST_hits_exp_og_genes, long_ph_orthogroups, by = c("sseqid_name" = "id")) %>%
      group_by(sseqid_name) %>%
      arrange(evalue, -bitscore, -pident) %>%
      slice(1) %>%
      ungroup() %>%
      arrange(evalue, -bitscore, -pident) %>%
      mutate(HOG = as.character(HOG),
             HOG = ifelse(is.na(HOG), paste("singleton", sep = "-", cumsum(is.na(HOG))),
                          as.character(HOG))) %>%
      group_by(HOG) %>%
      arrange(evalue, -bitscore, -pident) %>%
      slice(1) %>%
      ungroup() %>%
      arrange(evalue, -bitscore, -pident)
     

    #first row will should correspond to self match - the OG itself
    #self_and_closest_ogs

    #here the information provided by the user regarding max. additional OGs is used
    #suppose user chose max 5 additional orthogroups in config file
    add_OGs = 3


    #need to add check for number of additonal OGs/singletons
    #e.g. user sets max. add. OGs/singletons to 5 but we only can provide 4
    #+ have to consider the expanded OG itself, so:
    available_add_OGs <- nrow(self_and_closest_ogs) - 1 
    #available_add_OGs

    #in this case set add_OGs to max available
    if (available_add_OGs < add_OGs) {
      add_OGs <- available_add_OGs
    } 

    #empty list of precomputed size - add OGs plus the expanded OG itself
    add_OG_analysis_list <- vector(mode = "list", length = add_OGs+1)


    for (j in 1:(add_OGs+1)) {
      og_name <- self_and_closest_ogs[j,] %>%
      pull(HOG)
  
      #differnetiate between OG case and a singleton case - different handling:
      #for HOG get all associated genes/proteins from large long format table
      #for singletons get singelton gene/protein from the table itself
      if (!str_detect(og_name, "singleton") == TRUE) {
  
        name_curr_close_HOG <- self_and_closest_ogs[j,] %>%
                                 pull(HOG)
  
        ids_curr_close_HOG <- long_ph_orthogroups %>%
                                filter(HOG %in% name_curr_close_HOG) %>%
                                pull(id)
  
        add_OG_analysis_list[[j]] <- ids_curr_close_HOG
        names(add_OG_analysis_list)[j] <- name_curr_close_HOG
  
      } else {
        name_curr_close_singleton <- self_and_closest_ogs[j,] %>%
                                     pull(HOG)
  
        id_curr_close_HOG <- self_and_closest_ogs[j,] %>%
                               pull(sseqid_name)
  
        add_OG_analysis_list[[j]] <- id_curr_close_HOG
        names(add_OG_analysis_list)[j] <- name_curr_close_singleton
      }
    }
    
    #create copy of the list and remove the names
    #add all previous gene/proteins to the next OG
    # -> e.g. the former seventh OG list element will contain all genes/proteins of all lower numbered OGs
    #the final "cum_add_OG_analysis_list" will be a list with each next element having the cumulative set of all previous and own gene/protein ids 
    cum_add_OG_analysis_list <- add_OG_analysis_list
    names(cum_add_OG_analysis_list) <- NULL
    #copy_add_OG_analysis_list

    for (k in 0:(length(cum_add_OG_analysis_list)-1)) {
      cum_add_OG_analysis_list[[k+1]] <- unlist(c(cum_add_OG_analysis_list[k], cum_add_OG_analysis_list[k+1]))
    }

    #lapply(cum_add_OG_analysis_list, length)
    #print(i)
    #print(cum_add_OG_analysis_list)
    
    
    #create directory/ies - for each HOG under "add_OGs_sets"
    #dir.create(paste("tea/", num, "/add_OGs_sets/", i, sep = ""))
    dir.create(paste("../tea/", num, "/add_OGs_sets/", i, sep = ""))    
    
    #iterate over cum. list and write cumulative sets into seperate files
    for (l in 1:length(cum_add_OG_analysis_list)) {
      write_lines(cum_add_OG_analysis_list[[l]],
                  #paste("tea/", num, "/add_OGs_sets/", i, "/add_OGs_set_num-", l, ".txt", sep = "")
                  paste("../tea/", num, "/add_OGs_sets/", i, "/add_OGs_set_num-", l, ".txt", sep = "")
      )

    }

    #append current cum. list to summary list & name element after OG
    summary_add_OG_analysis_list[[loop_index]] <- cum_add_OG_analysis_list
    names(summary_add_OG_analysis_list)[loop_index] <- i
    
}


## in this final step we use the summary list to create S3-list nest structure which contains all info 
## of the summary but in the format we will embed in the final results object as part of the final_tea computation

#length of add OGs analysis summary list
summary_length <- length(summary_add_OG_analysis_list)

#create empty list of length of the summary list
add_og_complete_object <- vector(mode = "list", length = summary_length)

#iterate over all OGs in summary list
for (og in 1:summary_length) {
 
  #amount of sets for current OG - (necessary since less addtional OGs than max wished by user is possible)
  curr_og_n_sets <- length(summary_add_OG_analysis_list[[og]])
    
  #empty list with n elements = n sets for currently analyzed OG
  og_all_sets <- vector(mode = "list", length = curr_og_n_sets)
    
  for (set in 1:curr_og_n_sets) {

    curr_set <- new("add_OG_set",
               genes=as_tibble(
                       unlist(
                         summary_add_OG_analysis_list[[og]][set],
                       )  
                     ) 
        )
    curr_set <- list(curr_set)
    #names(curr_set) <- paste0(set)
  
    og_all_sets[set] <- curr_set
    names(og_all_sets)[set] <- paste0("set_", set)
      
  }
    
  curr_add_og <- new("add_OG_analysis",
                     add_OG_analysis=og_all_sets
                     )

  curr_add_og <- list(curr_add_og) 
 
  add_og_complete_object[og] <- curr_add_og

  names(add_og_complete_object)[og]  <- names(summary_add_OG_analysis_list[og])
}


# save summary table for aditional OG analysis to hypothesis specific ("num") RDS file
#saveRDS(add_og_complete_object, paste("tea/", num, "/add_OGs_sets/add_OG_analysis_object.RDS", sep = ""))
saveRDS(add_og_complete_object, paste("../tea/", num, "/add_OGs_sets/add_OG_analysis_object.RDS", sep = ""))

In [72]:
# nested snakemake checkpoints are annoying at the moment 
# quick fix - create empty addtional set_num files which we can ignore but nonetheless exist
add_OGs = 3

max_n_sets = add_OGs + 1

for (n in names(add_og_complete_object)) {
    
    og_n_sets <- length(
         list.files(path = paste0("../tea/2/add_OGs_sets/", n))
        )
    
    og_sets <- list.files(path = paste0("../tea/2/add_OGs_sets/", n))
    
    if (og_n_sets < max_n_sets) {
        print(n)
        print(og_sets)
        n_missing_sets <- max_n_sets - og_n_sets
        print(n_missing_sets)
        
        for (m in 1:n_missing_sets) {
          value <- max_n_sets - m + 1
          print(value)
          missing_file <- paste0("../tea/2/add_OGs_sets/", n, "/", "add_OGs_set_num-", value, ".txt")
          print(missing_file)
            
          file.create(missing_file)
        }
        #file.create("my.csv")
    }
    
}



[1] "N0.HOG0000049"
character(0)
[1] 4
[1] 4
[1] "../tea/2/add_OGs_sets/N0.HOG0000049/add_OGs_set_num-4.txt"


Warning message in file.create(missing_file):
“cannot create file '../tea/2/add_OGs_sets/N0.HOG0000049/add_OGs_set_num-4.txt', reason 'No such file or directory'”


[1] 3
[1] "../tea/2/add_OGs_sets/N0.HOG0000049/add_OGs_set_num-3.txt"


Warning message in file.create(missing_file):
“cannot create file '../tea/2/add_OGs_sets/N0.HOG0000049/add_OGs_set_num-3.txt', reason 'No such file or directory'”


[1] 2
[1] "../tea/2/add_OGs_sets/N0.HOG0000049/add_OGs_set_num-2.txt"


...

Warning message in file.create(missing_file):
“cannot create file '../tea/2/add_OGs_sets/N0.HOG0003758/add_OGs_set_num-3.txt', reason 'No such file or directory'”


[1] 2
[1] "../tea/2/add_OGs_sets/N0.HOG0003758/add_OGs_set_num-2.txt"


Warning message in file.create(missing_file):
“cannot create file '../tea/2/add_OGs_sets/N0.HOG0003758/add_OGs_set_num-2.txt', reason 'No such file or directory'”


[1] 1
[1] "../tea/2/add_OGs_sets/N0.HOG0003758/add_OGs_set_num-1.txt"


Warning message in file.create(missing_file):
“cannot create file '../tea/2/add_OGs_sets/N0.HOG0003758/add_OGs_set_num-1.txt', reason 'No such file or directory'”


### playaround

In [25]:
add_og_complete_object$N0.HOG0003790@add_OG_analysis$set_1

An object of class "add_OG_set"
Slot "genes":
# A tibble: 4 × 1
  value              
  <chr>              
1 AT2G44080.1        
2 HORVU2Hr1G077510.2 
3 HORVU2Hr1G077510.1 
4 Zm00001d007280_P001


In [103]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("treeio")
#BiocManager::install("Biostrings")

#library(treeio)
#library(Biostrings)

#extend expanded_OG (hypothesis-OG specific) class with "list" element "add_OG_analysis"

#under "add_OG_analysis" - we have n integer values (representing the number of addtional OGs -1; 1 = no addtional OGs since expanded OG itself)

# class for extended BLAST hits info
#setClass("add_OG_set", 
#         slots=list(genes="tbl_df",
#                    tree="phylo",
#                    msa="AAStringSet")
#         )

#setClass("complete_add_OG_analysis", 
#         slots=list(complete_add_OG_analysis="list")
#         )

setClass("add_OG_analysis", 
         slots=list(add_OG_analysis="list")
         )

setClass("add_OG_set", 
         slots=list(genes="tbl_df")
         )

In [183]:
#length of add OGs analysis summary list
summary_length <- length(summary_add_OG_analysis_list)

#create empty list of length of the summary list
add_og_complete_object <- vector(mode = "list", length = summary_length)

#iterate over all OGs in summary list
for (og in 1:summary_length) {
 
  #amount of sets for current OG - (necessary since less addtional OGs than max wished by user is possible)
  curr_og_n_sets <- length(summary_add_OG_analysis_list[[og]])
    
  #empty list with n elements = n sets for currently analyzed OG
  og_all_sets <- vector(mode = "list", length = curr_og_n_sets)
    
  for (set in 1:curr_og_n_sets) {

    curr_set <- new("add_OG_set",
               genes=as_tibble(
                       unlist(
                         summary_add_OG_analysis_list[[og]][set],
                       )  
                     ) 
        )
    curr_set <- list(curr_set)
    #names(curr_set) <- paste0(set)
  
    og_all_sets[set] <- curr_set
    names(og_all_sets)[set] <- paste0("set_", set)
      
  }
    
  curr_add_og <- new("add_OG_analysis",
                     add_OG_analysis=og_all_sets
                     )

  curr_add_og <- list(curr_add_og) 
 
  add_og_complete_object[og] <- curr_add_og

  names(add_og_complete_object)[og]  <- names(summary_add_OG_analysis_list[og])
}


In [188]:
str(add_og_complete_object$N0.HOG0000011@add_OG_analysis)

List of 4
 $ set_1:Formal class 'add_OG_set' [package ".GlobalEnv"] with 1 slot
  .. ..@ genes: tibble [68 × 1] (S3: tbl_df/tbl/data.frame)
  .. .. ..$ value: chr [1:68] "AT2G11035.1" "AT2G02520.1" "AT2G22440.1" "HORVU2Hr1G059140.63" ...
 $ set_2:Formal class 'add_OG_set' [package ".GlobalEnv"] with 1 slot
  .. ..@ genes: tibble [112 × 1] (S3: tbl_df/tbl/data.frame)
  .. .. ..$ value: chr [1:112] "AT2G11035.1" "AT2G02520.1" "AT2G22440.1" "HORVU2Hr1G059140.63" ...
 $ set_3:Formal class 'add_OG_set' [package ".GlobalEnv"] with 1 slot
  .. ..@ genes: tibble [123 × 1] (S3: tbl_df/tbl/data.frame)
  .. .. ..$ value: chr [1:123] "AT2G11035.1" "AT2G02520.1" "AT2G22440.1" "HORVU2Hr1G059140.63" ...
 $ set_4:Formal class 'add_OG_set' [package ".GlobalEnv"] with 1 slot
  .. ..@ genes: tibble [130 × 1] (S3: tbl_df/tbl/data.frame)
  .. .. ..$ value: chr [1:130] "AT2G11035.1" "AT2G02520.1" "AT2G22440.1" "HORVU2Hr1G059140.63" ...
